In [ ]:
import tensorflow.compat.v1 as tf
from tensorflow import keras
import numpy as np
import random
import os
import gc
import time
import platform

In [ ]:
tf.disable_v2_behavior()

from tensorflow.compat.v1.keras import backend as K
sess = tf.Session()
K.set_session(sess)

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
NUM_CLASSES = 10
WIDTH = 32
NUM_CHANNELS = 3

DATASET = "cifar10"
BASE_DIR = f""

TRIAL = 1
MODEL_DIR = f"{BASE_DIR}/tmp/cifar10/original/{TRIAL}/model_ckpt_11.h5"

# Data loading

In [ ]:
def _normalize(X):
  assert X.dtype == np.uint8
  X = X.astype(np.float64)
  X /= 255
  return X

def get_one_hot(targets, nb_classes):
  res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
  return res.reshape(list(targets.shape)+[nb_classes])

def load_standard_cifar10():
  (X_train, Y_train), (X_validation, Y_validation) = tf.keras.datasets.cifar10.load_data()
  X_train = X_train.reshape(X_train.shape[0], WIDTH, WIDTH, NUM_CHANNELS)
  X_validation = X_validation.reshape(X_validation.shape[0], WIDTH, WIDTH, NUM_CHANNELS)

  X_train = _normalize(X_train)
  X_validation = _normalize(X_validation)

  Y_train = Y_train.astype(np.int32)
  Y_validation = Y_validation.astype(np.int32)

  return X_train, Y_train, X_validation, Y_validation

def load_cifar10_train_dev(num_dev=100):
  # randomly select and fixed for future (tracin-like strategy but their indices available only for mnist)
  # selected_dev = np.random.randint(0, X_validation.shape[0], num_dev)
  selected_dev = [5214, 2304, 5947, 9428, 2717, 8296, 7736, 8291, 5235, 54,
                  7499, 9590, 3675, 1932, 6646, 8719, 6484, 6306, 3066, 2442,
                  6106, 1949, 4320,  541, 1318, 5967, 2773, 3847, 1152, 9937,
                  7469, 5982, 7644, 5820, 8152, 9518,  601, 3953, 4931, 1924,
                  5342, 5467, 6718, 6779, 2860, 2440, 5480, 1178,  222, 7909,
                  6394, 3511, 8729, 6261, 7192, 9453, 5257, 9077, 6419, 3280,
                  3725, 3601, 8174, 5703, 4954, 9536, 4783, 2234, 7365, 2405,
                  3073, 2780, 7461, 3525, 7573, 6764, 9962, 7527,  992,  315,
                  6260, 9061,  592, 8003, 7594, 1930, 7215, 5124, 7531, 9471,
                  2824, 3533, 6062, 3946, 5246, 4440,  414, 3572, 4899, 884]
  X_train, Y_train, X_validation, Y_validation = load_standard_cifar10()
  X_dev = X_validation[selected_dev]
  Y_dev = Y_validation[selected_dev]
  return X_train, Y_train, X_dev, Y_dev

In [ ]:
X_train, Y_train, X_dev, Y_dev = load_cifar10_train_dev()
Y_train = get_one_hot(Y_train, NUM_CLASSES)
Y_dev = get_one_hot(Y_dev, NUM_CLASSES)

170508288/170498071 [==============================] - 4s 0us/step


In [ ]:
Y_train = np.squeeze(Y_train)
Y_dev = np.squeeze(Y_dev)

# Model loading

In [ ]:
model = keras.models.load_model(MODEL_DIR)
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 conv2d_7 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 16, 16, 32)       0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 16, 16, 32)        0         
                                                                 
 conv2d_8 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 conv2d_9 (Conv2D)           (None, 16, 16, 64)        36928     
                                                      

# Influence functions implemention

In [ ]:
from tensorflow.compat.v1.keras import layers


def flatten(tensor_list):
  """Flattens a list of tensor into a single rank-1 tensor."""
  return tf.concat([tf.reshape(x, [-1]) for x in tensor_list], 0)


class InfSketchLayer(layers.Layer):
  """A Keras layer encapsulating the influence computation."""

  def __init__(self,
               base_params_fn,
               base_model_num_params,
               target=None,
               sketch_size=100):
    """Constructs the InfSketchLayer.

    Args:
      base_params_fn: A callable returning a list of the base model parameters.
      base_model_num_params: The total number of base model parameters.
      target: An optional matrix of shape (sketch_size, base_model_num_params)
        that will be the target matrix for the sketch computation. The sketch
        computed will be Hessian^{-1} * target.
      sketch_size: the size of the sketch. Larger values give
        better accuracy but involve slower computation.
    """
    super(InfSketchLayer, self).__init__()
    self._sketch_size = sketch_size
    self._base_params_fn = base_params_fn
    if target is None:
      self.target = self.add_weight(
          name='target',
          shape=(self._sketch_size, base_model_num_params),
          dtype=tf.float32,
          initializer=tf.keras.initializers.random_normal(stddev=1.0 /
                                                          np.sqrt(sketch_size)),
          trainable=False)
    else:
      self.target = target
      self._sketch_size = target.shape[0]
    self.sketch = self.add_weight(
        name='sketch',
        shape=self.target.shape,
        initializer='zeros',
        trainable=True)

  def call(self, inputs, training=True):
    """inputs is expected to be the gradient of the loss."""
    if training:
      grad1_x_sketch = tf.matmul(self.sketch, tf.reshape(inputs[0], [-1, 1]))
      grad2_x_sketch = tf.matmul(self.sketch, tf.reshape(inputs[1], [-1, 1]))
      base_params = self._base_params_fn()

      def loss_i(i):
        # si = self.sketch[i]
        ti = self.target[i]
        gs1 = grad1_x_sketch[i][0]
        hs1 = tf.cast(flatten(tf.gradients(gs1, base_params)), tf.float32)
        gs2 = grad2_x_sketch[i][0]
        hs2 = tf.cast(flatten(tf.gradients(gs2, base_params)), tf.float32)
        return tf.tensordot(hs1 - ti, hs2 - ti, 1)

      _, loss = tf.while_loop(
          lambda i, _: i < self._sketch_size,
          lambda i, loss: (i + 1, loss + loss_i(i)),
          [tf.constant(0, tf.int32),
           tf.constant(0.0, tf.float32)])
      output = tf.reshape(loss, [1, -1])
    else:
      grad_x_sketch = tf.matmul(self.sketch, inputs)
      output = tf.matmul(tf.transpose(self.target), grad_x_sketch)
    return output


class LossGradientLayer(layers.Layer):
  """A Keras layer computing the gradient of the loss."""

  def __init__(self, base_model_fn):
    super(LossGradientLayer, self).__init__()
    self._base_model_fn = base_model_fn

  def call(self, inputs):
    base_loss, base_params = self._base_model_fn(inputs)
    return tf.cast(flatten(tf.gradients(base_loss, base_params)), tf.float32)


def _identity_loss(_, y_pred):
  """Defines the identity loss function."""
  return y_pred


def _base_model_fns(base_model, loss_fn):
  """Helper function to construct base_model_fn and base_params_fn."""
  def base_model_fn(inputs):
    features = inputs[0]
    labels = inputs[1]
    preds = base_model(features, training=False)
    return loss_fn(labels, preds), base_model.trainable_variables

  def base_params_fn():
    return base_model.trainable_variables

  return base_model_fn, base_params_fn


def test_loss_grads(base_model, loss_fn, test_inputs):
  """Computes the loss gradient at the given test points."""
  num_points = test_inputs[0].shape[0]
  features_input = tf.keras.Input(shape=test_inputs[0].shape[1:])
  labels_input = tf.keras.Input(shape=test_inputs[1].shape[1:])
  base_model_fn, _ = _base_model_fns(base_model, loss_fn)
  test_grad = tf.reshape(
      LossGradientLayer(base_model_fn)([features_input, labels_input]), [1, -1])
  m = tf.keras.Model(inputs=[features_input, labels_input], outputs=test_grad)
  grads = np.zeros((num_points, base_model.count_params()), dtype=np.float32)
  for i in range(num_points):
    grads[i] = m.predict([[test_inputs[0][i]], [test_inputs[1][i]]])[0]
  return grads


class InfSketcher(object):
  """A helper class encapsulating two models: sketcher_model and inf_model.

  sketcher_model: computes the inverse Hessian sketch
  inf_model: computes influences, accessed via the inf_model() function.
  """

  def __init__(self,
               base_model,
               inputs,
               loss_fn,
               target=None,
               sketch_size=100,
               optimizer='sgd',
               sketch_path=None):
    base_model_fn, base_params_fn = _base_model_fns(base_model, loss_fn)
    self._grad_layer = LossGradientLayer(base_model_fn)
    f0, f1 = tf.split(inputs[0], 2)
    l0, l1 = tf.split(inputs[1], 2)
    grad1 = self._grad_layer([f0, l0])
    grad2 = self._grad_layer([f1, l1])
    self._is_layer = InfSketchLayer(base_params_fn, base_model.count_params(),
                                    target, sketch_size)
    train_loss = self._is_layer([grad1, grad2], training=True)
    self.sketcher_model = tf.keras.Model(
        inputs=inputs, outputs=train_loss)
    self.sketcher_model.compile(optimizer=optimizer, loss=_identity_loss)
    if sketch_path is not None:
      self.sketcher_model.load_weights(sketch_path)

  def get_inf_model(self, test_input, train_inputs):
    """Influence computing model using a single test point as target."""
    inf_grad = self._grad_layer(train_inputs)
    test_grad = self._grad_layer(test_input)
    invhess_x_test_grad = self._is_layer(
        tf.reshape(test_grad, [-1, 1]), training=False)
    influence = tf.reshape(tf.tensordot(inf_grad, invhess_x_test_grad, 1), [-1])
    self.inf_model = tf.keras.Model(
        inputs=train_inputs, outputs=influence)
    return self.inf_model

  def get_grads_inf_model(self, test_grads, train_inputs):
    """Influence computing model for target/test grads using the sketched inv hessian."""

    inf_grad = self._grad_layer(train_inputs)  # shape = [num_params]
    invhess_x_test_grads = self._is_layer(
        tf.transpose(test_grads),
        training=False)  # shape = [num_params, num_test_inputs]
    # influence of this training point over the test_grads provided.
    influence = tf.reshape(
        tf.matmul(tf.reshape(inf_grad, [1, -1]), invhess_x_test_grads),
        [1, -1])  # shape = [num_test_inputs]
    self.inf_model = tf.keras.Model(inputs=train_inputs, outputs=influence)
    return self.inf_model

  def target_inf_model(self, train_inputs):
    inf_grad = self._grad_layer(train_inputs)
    sketch_x_grad = tf.reshape(
        tf.matmul(self._is_layer.sketch, tf.reshape(inf_grad, [-1, 1])),
        [1, -1])
    self.target_inf_model = tf.keras.Model(
        inputs=train_inputs, outputs=sketch_x_grad)
    return self.target_inf_model

  def save(self, path):
    self.sketcher_model.save_weights(path)

# Influence on loss of test points

In [ ]:
test_points = [X_dev, Y_dev]
test_grads = test_loss_grads(model, tf.keras.losses.CategoricalCrossentropy(), test_points)
test_grads = tf.constant(test_grads)

In [ ]:
features_input = tf.keras.Input(shape=(WIDTH, WIDTH, NUM_CHANNELS))
labels_input = tf.keras.Input(shape=(NUM_CLASSES,))

In [ ]:
sketch_path = BASE_DIR + '/tmp/cifar10/original/{0}/sketcher_model.h5'.format(TRIAL)
if os.path.exists(sketch_path):
  print('using sketch from {0}'.format(sketch_path))
  my_is = InfSketcher(model,
                                [features_input, labels_input],
                                tf.keras.losses.CategoricalCrossentropy(),
                                target=test_grads,
                                optimizer='adam',
                                sketch_path=sketch_path,
                      sketch_size=20)  # temp; to account for memory
  sketcher_model = my_is.sketcher_model
else:
  print('sketch_path file does not exist, training a sketch')
  my_is = InfSketcher(model,
                                [features_input, labels_input],
                                tf.keras.losses.CategoricalCrossentropy(),
                                optimizer='adam',
                                target=test_grads)
  sketcher_model = my_is.sketcher_model

sketch_path file does not exist, training a sketch


In [ ]:
for i in range(8):
  print('round {}'.format(i))
  # sketcher_model.fit(x=[X_train, Y_train], y=None, verbose=1, epochs=10, batch_size=10000)
  sketcher_model.fit(x=[X_train, Y_train], y=None, verbose=1, epochs=5, batch_size=32)
  sketcher_model.save_weights(sketch_path)
print('saved sketch to {0}'.format(sketch_path))

In [ ]:
my_is.sketcher_model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 input_8 (InputLayer)           [(None, 10)]         0           []                               
                                                                                                  
 tf_op_layer_split_6 (TensorFlo  [(None, 32, 32, 3),  0          ['input_7[0][0]']                
 wOpLayer)                       (None, 32, 32, 3)]                                               
                                                                                                  
 tf_op_layer_split_7 (TensorFlo  [(None, 10),        0           ['input_8[0][0]']          

In [ ]:
NUM_TRAIN = Y_train.shape[0]
NUM_DEV = Y_dev.shape[0]

In [ ]:
target_inf_model = my_is.target_inf_model([features_input, labels_input])
# influences = np.zeros(shape=(60000,100), dtype=np.float32)
influences = np.zeros(shape=(NUM_TRAIN,NUM_DEV), dtype=np.float32)
for i in range(NUM_TRAIN):
  if (i % 10000) == 0:
    print(i)
  influences[i] = my_is.target_inf_model.predict([[X_train[i]], [Y_train[i]]])[0]
# np.savez_compressed('/tmp/ckpts/{0}/dev_influences_inf_fun.npz'.format(TRIAL), influences)
np.savez_compressed(BASE_DIR + '/tmp/cifar10/original/{0}/dev_influences_inf_fun.npz'.format(TRIAL), influences)

0
10000
20000
30000
40000
